In [1]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, IntSlider, ToggleButtons
from tensorflow.keras.models import load_model
import albumentations as A

# Custom objects required for loading the U-Net segmentation models
import sys 
sys.path.append('..')
from segmentation_losses import (
    dice_coefficient_binary,
    dice_loss_binary,
    log_cosh_dice_loss_binary,
    iou_binary
)

In [2]:
model = load_model(
    "../trained_models/task02_heart_unet_3d_segmentation_model.h5",
    custom_objects={
        "log_cosh_dice_loss_binary": log_cosh_dice_loss_binary,
        "dice_coefficient_binary": dice_coefficient_binary,
        "iou_binary": iou_binary
    }
)

In [3]:
def standardize(mri):
    """
    Standardize mean and standard deviation of each channel and z_dimension slice to mean 0 and standard 
     deviation 1.

    Note: setting the type of the input mri to np.float16 beforehand causes issues, set it afterwards.

    Args:
        mri (np.array): input mri, shape (dim_x, dim_y, dim_z, num_channels)
    Returns:
        standardized_mri (np.array): standardized version of input mri
    """

    standardized_mri = np.zeros(mri.shape)

    # Iterate over channels
    for c in range(mri.shape[3]):
        # Iterate over the `z` depth dimension
        for z in range(mri.shape[2]):
            # Get a slice of the mri at channel c and z-th dimension
            mri_slice = mri[:, :, z, c]

            # Subtract the mean from mri_slice
            centered = mri_slice - np.mean(mri_slice)

            # Divide by the standard deviation (only if it is different from zero)
            if np.std(centered) != 0:
                centered_scaled = centered / np.std(centered)

                # Update the slice of standardized mri with the centered and scaled mri
                standardized_mri[:, :, z, c] = centered_scaled

    return standardized_mri

### 1. Visualizing predictions on validation set images

### 1.1. Visualizing prediction on la_022 file

### 1.1.1 Chunk 1

In [4]:
mri_chunk1 = nib.load("../datasets/Task02_Heart_320x320x96/val/images/la_022_0.nii.gz").get_fdata().astype(np.float32)

In [5]:
mask_chunk1 = nib.load("../datasets/Task02_Heart_320x320x96/val/masks/la_022_0.nii.gz").get_fdata().astype(np.uint8)

In [6]:
mask_chunk1.shape

(320, 320, 96)

In [7]:
mri_standardized_chunk1 = standardize(mri_chunk1.reshape((mri_chunk1.shape[0], mri_chunk1.shape[1], mri_chunk1.shape[2], 1)))

In [8]:
mri_standardized_chunk1 = np.expand_dims(mri_standardized_chunk1, axis=0)  # Keras models require an additional dimension of 'batch_size'

In [9]:
prediction_chunk1 = model.predict(mri_standardized_chunk1)

In [10]:
prediction_chunk1 = np.squeeze(prediction_chunk1, axis=0)  # Remove 'batch_size' dimension

In [11]:
prediction_chunk1.shape

(320, 320, 96, 1)

In [12]:
prediction_chunk1

array([[[[3.5335725e-05],
         [3.5335725e-05],
         [3.5335725e-05],
         ...,
         [3.5335725e-05],
         [3.1880227e-05],
         [3.1585627e-05]],

        [[3.5335725e-05],
         [3.5335725e-05],
         [3.5114321e-05],
         ...,
         [3.5069679e-05],
         [2.8724477e-05],
         [3.0104733e-05]],

        [[3.5335725e-05],
         [3.5335725e-05],
         [3.4995479e-05],
         ...,
         [3.0953172e-05],
         [2.7633230e-05],
         [3.0193603e-05]],

        ...,

        [[3.5335725e-05],
         [3.6390113e-05],
         [3.6694335e-05],
         ...,
         [3.4164597e-05],
         [2.4771429e-05],
         [2.5988789e-05]],

        [[3.5335725e-05],
         [3.5335725e-05],
         [3.4669414e-05],
         ...,
         [3.1673164e-05],
         [2.6956264e-05],
         [2.9094010e-05]],

        [[3.5335725e-05],
         [3.5335725e-05],
         [3.1770269e-05],
         ...,
         [2.7075161e-05],
        

In [13]:
type(prediction_chunk1)

numpy.ndarray

In [14]:
prediction_chunk1 = (prediction_chunk1 >= 0.5).astype(np.uint8)  # Apply "Sigmoid" function on predicted probabilities per voxel

In [15]:
prediction_chunk1.shape

(320, 320, 96, 1)

In [16]:
prediction_chunk1 = prediction_chunk1.reshape((prediction_chunk1.shape[0], prediction_chunk1.shape[1], prediction_chunk1.shape[2]))

In [17]:
prediction_chunk1.shape

(320, 320, 96)

In [18]:
np.unique(prediction_chunk1)

array([0, 1], dtype=uint8)

#### Model prediction vs groundtruth number of labels

In [19]:
# Number of labels per class in model prediction
print("Background - class 0")
print(len(prediction_chunk1[prediction_chunk1 == 0])) 
print("--------")
print("Left Atrium - class 1")
print(len(prediction_chunk1[prediction_chunk1 == 1]))

Background - class 0
9792279
--------
Left Atrium - class 1
38121


In [20]:
# Number of labels per class in ground truth mask
print("Background - class 0")
print(len(mask_chunk1[mask_chunk1 == 0])) 
print("--------")
print("Left Atrium - class 1")
print(len(mask_chunk1[mask_chunk1 == 1]))

Background - class 0
9797268
--------
Left Atrium - class 1
33132


In [21]:
# Create layer slider
select_layer = IntSlider(min=0, max=mri_chunk1.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(layer):
    print(f"Plotting Layer: {layer}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 3, 1)
    plt.title("MRI la_022", fontsize=20)
    plt.imshow(mri_chunk1[:, :, layer], cmap='gray');
    plt.axis('off')
    
    fig.add_subplot(1, 3, 2)
    plt.title("Groundtruth mask", fontsize=20)
    plt.imshow(mask_chunk1[:, :, layer], cmap='gray')
    plt.axis('off');
    
    fig.add_subplot(1, 3, 3)
    plt.title("Model prediction mask", fontsize=20)
    plt.imshow(prediction_chunk1[:, :, layer], cmap='gray')
    plt.axis('off');
 
    
# Use the interactive() tool to create the visualization
interactive(plot, layer=select_layer)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Select Layer', max=95), Output(…

### 1.1.2 Chunk 2

In [22]:
mri_chunk2 = nib.load("../datasets/Task02_Heart_320x320x96/val/images/la_022_1.nii.gz").get_fdata().astype(np.float32)

In [23]:
mask_chunk2 = nib.load("../datasets/Task02_Heart_320x320x96/val/masks/la_022_1.nii.gz").get_fdata().astype(np.uint8)

In [24]:
mask_chunk2.shape

(320, 320, 96)

In [25]:
mri_standardized_chunk2 = standardize(mri_chunk2.reshape((mri_chunk2.shape[0], mri_chunk2.shape[1], mri_chunk2.shape[2], 1)))

In [26]:
mri_standardized_chunk2 = np.expand_dims(mri_standardized_chunk2, axis=0)  # Keras models require an additional dimension of 'batch_size'

In [27]:
prediction_chunk2 = model.predict(mri_standardized_chunk2)

In [28]:
prediction_chunk2 = np.squeeze(prediction_chunk2, axis=0)  # Remove 'batch_size' dimension

In [29]:
prediction_chunk2.shape

(320, 320, 96, 1)

In [30]:
prediction_chunk2

array([[[[3.5335725e-05],
         [3.5335725e-05],
         [3.5335725e-05],
         ...,
         [3.5335725e-05],
         [3.5335725e-05],
         [3.5335725e-05]],

        [[3.5335725e-05],
         [3.5335725e-05],
         [3.4836383e-05],
         ...,
         [3.5335725e-05],
         [3.5335725e-05],
         [3.5335725e-05]],

        [[3.5335725e-05],
         [3.4574328e-05],
         [3.4647899e-05],
         ...,
         [3.5335725e-05],
         [3.5335725e-05],
         [3.5335725e-05]],

        ...,

        [[3.5335725e-05],
         [3.6739853e-05],
         [3.6926933e-05],
         ...,
         [3.5335725e-05],
         [3.5335725e-05],
         [3.5335725e-05]],

        [[3.5335725e-05],
         [3.5335725e-05],
         [3.4710240e-05],
         ...,
         [3.5335725e-05],
         [3.5335725e-05],
         [3.5335725e-05]],

        [[3.5335725e-05],
         [3.5335725e-05],
         [3.1545231e-05],
         ...,
         [3.5335725e-05],
        

In [31]:
type(prediction_chunk2)

numpy.ndarray

In [32]:
prediction_chunk2 = (prediction_chunk2 >= 0.5).astype(np.uint8)  # Apply "Sigmoid" function on predicted probabilities per voxel

In [33]:
prediction_chunk2.shape

(320, 320, 96, 1)

In [34]:
prediction_chunk2 = prediction_chunk2.reshape((prediction_chunk2.shape[0], prediction_chunk2.shape[1], prediction_chunk2.shape[2]))

In [35]:
prediction_chunk2.shape

(320, 320, 96)

In [36]:
np.unique(prediction_chunk2)

array([0, 1], dtype=uint8)

#### Model prediction vs groundtruth number of labels

In [37]:
# Number of labels per class in model prediction
print("Background - class 0")
print(len(prediction_chunk2[prediction_chunk2 == 0])) 
print("--------")
print("Left Atrium - class 1")
print(len(prediction_chunk2[prediction_chunk2 == 1]))

Background - class 0
9830352
--------
Left Atrium - class 1
48


In [38]:
# Number of labels per class in ground truth mask
print("Background - class 0")
print(len(mask_chunk2[mask_chunk2 == 0])) 
print("--------")
print("Left Atrium - class 1")
print(len(mask_chunk2[mask_chunk2 == 1]))

Background - class 0
9829981
--------
Left Atrium - class 1
419


In [39]:
# Create layer slider
select_layer = IntSlider(min=0, max=mri_chunk2.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(layer):
    print(f"Plotting Layer: {layer}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 3, 1)
    plt.title("MRI la_022", fontsize=20)
    plt.imshow(mri_chunk2[:, :, layer], cmap='gray');
    plt.axis('off')
    
    fig.add_subplot(1, 3, 2)
    plt.title("Groundtruth mask", fontsize=20)
    plt.imshow(mask_chunk2[:, :, layer], cmap='gray')
    plt.axis('off');
    
    fig.add_subplot(1, 3, 3)
    plt.title("Model prediction mask", fontsize=20)
    plt.imshow(prediction_chunk2[:, :, layer], cmap='gray')
    plt.axis('off');
 
    
# Use the interactive() tool to create the visualization
interactive(plot, layer=select_layer)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Select Layer', max=95), Output(…

### 1.2. Visualizing prediction on la_023 file

### 1.1.1 Chunk 1

In [40]:
mri_chunk1 = nib.load("../datasets/Task02_Heart_320x320x96/val/images/la_023_0.nii.gz").get_fdata().astype(np.float32)

In [41]:
mask_chunk1 = nib.load("../datasets/Task02_Heart_320x320x96/val/masks/la_023_0.nii.gz").get_fdata().astype(np.uint8)

In [42]:
mask_chunk1.shape

(320, 320, 96)

In [43]:
mri_standardized_chunk1 = standardize(mri_chunk1.reshape((mri_chunk1.shape[0], mri_chunk1.shape[1], mri_chunk1.shape[2], 1)))

In [44]:
mri_standardized_chunk1 = np.expand_dims(mri_standardized_chunk1, axis=0)  # Keras models require an additional dimension of 'batch_size'

In [45]:
prediction_chunk1 = model.predict(mri_standardized_chunk1)

In [46]:
prediction_chunk1 = np.squeeze(prediction_chunk1, axis=0)  # Remove 'batch_size' dimension

In [47]:
prediction_chunk1.shape

(320, 320, 96, 1)

In [48]:
prediction_chunk1

array([[[[3.5335725e-05],
         [3.5335725e-05],
         [3.5335725e-05],
         ...,
         [3.5166933e-05],
         [3.1231088e-05],
         [3.0975436e-05]],

        [[3.5335725e-05],
         [3.5335725e-05],
         [3.5335725e-05],
         ...,
         [3.4971188e-05],
         [2.7791961e-05],
         [2.9899476e-05]],

        [[3.5335725e-05],
         [3.5335725e-05],
         [3.5335725e-05],
         ...,
         [3.2262316e-05],
         [2.6313590e-05],
         [3.0164825e-05]],

        ...,

        [[3.5335725e-05],
         [3.5335725e-05],
         [3.4731627e-05],
         ...,
         [3.5255514e-05],
         [2.5194789e-05],
         [2.4998204e-05]],

        [[3.5335725e-05],
         [3.5335725e-05],
         [3.5335725e-05],
         ...,
         [3.3086162e-05],
         [2.7738102e-05],
         [2.7977383e-05]],

        [[3.5335725e-05],
         [3.5335725e-05],
         [3.4795379e-05],
         ...,
         [2.6275426e-05],
        

In [49]:
type(prediction_chunk1)

numpy.ndarray

In [50]:
prediction_chunk1 = (prediction_chunk1 >= 0.5).astype(np.uint8)  # Apply "Sigmoid" function on predicted probabilities per voxel

In [51]:
prediction_chunk1.shape

(320, 320, 96, 1)

In [52]:
prediction_chunk1 = prediction_chunk1.reshape((prediction_chunk1.shape[0], prediction_chunk1.shape[1], prediction_chunk1.shape[2]))

In [53]:
prediction_chunk1.shape

(320, 320, 96)

In [54]:
np.unique(prediction_chunk1)

array([0, 1], dtype=uint8)

#### Model prediction vs groundtruth number of labels

In [55]:
# Number of labels per class in model prediction
print("Background - class 0")
print(len(prediction_chunk1[prediction_chunk1 == 0])) 
print("--------")
print("Left Atrium - class 1")
print(len(prediction_chunk1[prediction_chunk1 == 1]))

Background - class 0
9794583
--------
Left Atrium - class 1
35817


In [56]:
# Number of labels per class in ground truth mask
print("Background - class 0")
print(len(mask_chunk1[mask_chunk1 == 0])) 
print("--------")
print("Left Atrium - class 1")
print(len(mask_chunk1[mask_chunk1 == 1]))

Background - class 0
9787196
--------
Left Atrium - class 1
43204


In [57]:
# Create layer slider
select_layer = IntSlider(min=0, max=mri_chunk1.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(layer):
    print(f"Plotting Layer: {layer}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 3, 1)
    plt.title("MRI la_023", fontsize=20)
    plt.imshow(mri_chunk1[:, :, layer], cmap='gray');
    plt.axis('off')
    
    fig.add_subplot(1, 3, 2)
    plt.title("Groundtruth mask", fontsize=20)
    plt.imshow(mask_chunk1[:, :, layer], cmap='gray')
    plt.axis('off');
    
    fig.add_subplot(1, 3, 3)
    plt.title("Model prediction mask", fontsize=20)
    plt.imshow(prediction_chunk1[:, :, layer], cmap='gray')
    plt.axis('off');
 
    
# Use the interactive() tool to create the visualization
interactive(plot, layer=select_layer)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Select Layer', max=95), Output(…

### 1.1.2 Chunk 2

In [58]:
mri_chunk2 = nib.load("../datasets/Task02_Heart_320x320x96/val/images/la_023_1.nii.gz").get_fdata().astype(np.float32)

In [59]:
mask_chunk2 = nib.load("../datasets/Task02_Heart_320x320x96/val/masks/la_023_1.nii.gz").get_fdata().astype(np.uint8)

In [60]:
mask_chunk2.shape

(320, 320, 96)

In [61]:
mri_standardized_chunk2 = standardize(mri_chunk2.reshape((mri_chunk2.shape[0], mri_chunk2.shape[1], mri_chunk2.shape[2], 1)))

In [62]:
mri_standardized_chunk2 = np.expand_dims(mri_standardized_chunk2, axis=0)  # Keras models require an additional dimension of 'batch_size'

In [63]:
prediction_chunk2 = model.predict(mri_standardized_chunk2)

In [64]:
prediction_chunk2 = np.squeeze(prediction_chunk2, axis=0)  # Remove 'batch_size' dimension

In [65]:
prediction_chunk2.shape

(320, 320, 96, 1)

In [66]:
prediction_chunk2

array([[[[3.5335725e-05],
         [3.5335725e-05],
         [3.2904729e-05],
         ...,
         [3.5335725e-05],
         [3.5335725e-05],
         [3.5335725e-05]],

        [[3.5335725e-05],
         [3.4261531e-05],
         [3.2444659e-05],
         ...,
         [3.5335725e-05],
         [3.5335725e-05],
         [3.5335725e-05]],

        [[3.5335725e-05],
         [3.2231135e-05],
         [3.2539694e-05],
         ...,
         [3.5335725e-05],
         [3.5335725e-05],
         [3.5335725e-05]],

        ...,

        [[3.5335725e-05],
         [4.0659088e-05],
         [3.9590424e-05],
         ...,
         [3.5335725e-05],
         [3.5335725e-05],
         [3.5335725e-05]],

        [[3.5335725e-05],
         [3.7610433e-05],
         [3.5924277e-05],
         ...,
         [3.5335725e-05],
         [3.5335725e-05],
         [3.5335725e-05]],

        [[3.5335725e-05],
         [3.3400072e-05],
         [3.1492033e-05],
         ...,
         [3.5335725e-05],
        

In [67]:
type(prediction_chunk2)

numpy.ndarray

In [68]:
prediction_chunk2 = (prediction_chunk2 >= 0.5).astype(np.uint8)  # Apply "Sigmoid" function on predicted probabilities per voxel

In [69]:
prediction_chunk2.shape

(320, 320, 96, 1)

In [70]:
prediction_chunk2 = prediction_chunk2.reshape((prediction_chunk2.shape[0], prediction_chunk2.shape[1], prediction_chunk2.shape[2]))

In [71]:
np.unique(prediction_chunk2)

array([0], dtype=uint8)

#### Model prediction vs groundtruth number of labels

In [72]:
# Number of labels per class in model prediction
print("Background - class 0")
print(len(prediction_chunk2[prediction_chunk2 == 0])) 
print("--------")
print("Left Atrium - class 1")
print(len(prediction_chunk2[prediction_chunk2 == 1]))

Background - class 0
9830400
--------
Left Atrium - class 1
0


In [73]:
# Number of labels per class in ground truth mask
print("Background - class 0")
print(len(mask_chunk2[mask_chunk2 == 0])) 
print("--------")
print("Left Atrium - class 1")
print(len(mask_chunk2[mask_chunk2 == 1]))

Background - class 0
9830400
--------
Left Atrium - class 1
0


In [74]:
# Create layer slider
select_layer = IntSlider(min=0, max=mri_chunk2.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(layer):
    print(f"Plotting Layer: {layer}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 3, 1)
    plt.title("MRI la_023", fontsize=20)
    plt.imshow(mri_chunk2[:, :, layer], cmap='gray');
    plt.axis('off')
    
    fig.add_subplot(1, 3, 2)
    plt.title("Groundtruth mask", fontsize=20)
    plt.imshow(mask_chunk2[:, :, layer], cmap='gray')
    plt.axis('off');
    
    fig.add_subplot(1, 3, 3)
    plt.title("Model prediction mask", fontsize=20)
    plt.imshow(prediction_chunk2[:, :, layer], cmap='gray')
    plt.axis('off');
 
    
# Use the interactive() tool to create the visualization
interactive(plot, layer=select_layer)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Select Layer', max=95), Output(…

### 2. Testing on the medical decathlon test images for Task 2

In [75]:
def generate_model_prediction(model, mri):
    mri_standardized = standardize(mri)
    mri_standardized = np.expand_dims(mri_standardized, axis=0)  # Keras models require an additional dimension of 'batch_size'

    prediction = model.predict(mri_standardized)
    prediction = np.squeeze(prediction, axis=0)  # Remove 'batch_size' dimension

    prediction = (prediction >= 0.5).astype(np.uint8)  # Apply "Sigmoid" function on predicted probabilities per voxel
    prediction = prediction.reshape(prediction.shape[0], prediction.shape[1], prediction.shape[2])  # Remove extra channel dimension

    return prediction

In [76]:
def generate_model_prediction_full_file_chunks(model, mri):
    file_layer_size = 96
    original_mri_image_height_size = mri.shape[0]
    original_mri_image_width_size = mri.shape[1]
    original_file_layer_size = mri.shape[2]

    num_chunks = int(np.ceil(original_file_layer_size / file_layer_size))
    prediction = np.zeros((original_mri_image_height_size, original_mri_image_width_size, original_file_layer_size))
    chunks = np.zeros((num_chunks, original_mri_image_height_size, original_mri_image_width_size, file_layer_size, 1))

    reshaped_mri = mri.reshape((original_mri_image_height_size, original_mri_image_width_size, original_file_layer_size, 1))

    # Split the file into chunks
    for i in range(num_chunks):
        if i == num_chunks - 1:  # last iteration (padding the last chunk with zeros and using it)
            chunks[i, :, :, :original_file_layer_size % file_layer_size, :] = reshaped_mri[:, :, i*file_layer_size: i*file_layer_size + (original_file_layer_size % file_layer_size), :]
        else:
            chunks[i, :, :, :, :] = reshaped_mri[:, :, i*file_layer_size: (i+1)*file_layer_size, :]

    # Generate predictions per chunk
    for i in range(num_chunks):
        if i == num_chunks - 1:  # Last chunk, only get the remainder of the layers
            prediction[:, :, i*file_layer_size:] = generate_model_prediction(model=model, mri=chunks[i, :, :, :])[:, :, :original_file_layer_size - (i*file_layer_size)]  # Returns type np.uint8
        else:
            prediction[:, :, i*file_layer_size: (i+1)*file_layer_size] = generate_model_prediction(model=model, mri=chunks[i, :, :, :])  # Returns type np.uint8
    
    return prediction

### 2.1 Visualizing prediction on la_002 file

In [77]:
mri = nib.load("../datasets/Task02_Heart/test_images_for_model_prediction_submission/la_002.nii.gz").get_fdata().astype(np.float32)

In [78]:
mri.shape

(320, 320, 140)

In [79]:
prediction = generate_model_prediction_full_file_chunks(model, mri)

In [80]:
prediction.shape

(320, 320, 140)

In [81]:
type(prediction)

numpy.ndarray

In [82]:
prediction = (prediction >= 0.5).astype(np.uint8)  # Apply "Sigmoid" function on predicted probabilities per voxel

In [83]:
prediction.shape

(320, 320, 140)

In [84]:
np.unique(prediction)

array([0, 1], dtype=uint8)

#### Model prediction number of labels (no available masks)

In [85]:
# Number of labels per class in model prediction
print("Background - class 0")
print(len(prediction[prediction == 0])) 
print("--------")
print("Left Atrium - class 1")
print(len(prediction[prediction == 1]))

Background - class 0
14310268
--------
Left Atrium - class 1
25732


In [86]:
# Create layer slider
select_layer = IntSlider(min=0, max=mri.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(layer):
    print(f"Plotting Layer: {layer}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 2, 1)
    plt.title("la_002", fontsize=20)
    plt.imshow(mri[:, :, layer], cmap='gray');
    plt.axis('off')
    
    fig.add_subplot(1, 2, 2)
    plt.title("Model prediction mask", fontsize=20)
    plt.imshow(prediction[:, :, layer], cmap='gray')
    plt.axis('off');

# Use the interactive() tool to create the visualization
interactive(plot, layer=select_layer)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Select Layer', max=139), Output…

### 2.2 Visualizing prediction on la_001 file (Original size bigger than model input size, predictions need to be resized to original size as well)

In [87]:
mri = nib.load("../datasets/Task02_Heart/test_images_for_model_prediction_submission/la_001.nii.gz").get_fdata().astype(np.float32)

In [88]:
mri.shape

(400, 400, 180)

In [89]:
transform = A.Compose(
    p=1.0,
    transforms=[
        A.Resize(
            height=320,
            width=320,
            interpolation=1,
            p=1
        )
    ]
)

transform_original_dimensions = A.Compose(
    p=1.0,
    transforms=[
        A.Resize(
            height=mri.shape[0],
            width=mri.shape[1],
            interpolation=1,
            p=1
        )
    ]
)

In [90]:
# Case 1: If there are more than one channel, Albumentations.Resize() would only work on each
#  channel separately, working on all dimensions at once would cause it to fail
if len(mri.shape) >= 4:
    temp_mri_channels = np.zeros((image_height_size, image_width_size, original_file_layer_size, mri.shape[3]))
    for i in range(int(mri.shape[3])):
        transformed = transform(image=mri[:, :, :, i])
        temp_mri = transformed["image"]
        temp_mri_channels[:, :, :, i] = temp_mri[:, :, :]

    mri = temp_mri_channels
else:
    # Case 2: One channel file, Albumentations.Resize() would work normally
    transformed = transform(image=mri)
    mri = transformed["image"]

In [91]:
mri.shape

(320, 320, 180)

In [92]:
prediction = generate_model_prediction_full_file_chunks(model, mri)

In [93]:
prediction.shape

(320, 320, 180)

In [94]:
type(prediction)

numpy.ndarray

In [95]:
prediction = (prediction >= 0.5).astype(np.uint8)  # Apply "Sigmoid" function on predicted probabilities per voxel

In [96]:
prediction.shape

(320, 320, 180)

In [97]:
np.unique(prediction)

array([0, 1], dtype=uint8)

#### Model prediction number of labels (no available masks)

In [98]:
# Number of labels per class in model prediction
print("Background - class 0")
print(len(prediction[prediction == 0])) 
print("--------")
print("Left Atrium - class 1")
print(len(prediction[prediction == 1]))

Background - class 0
18398595
--------
Left Atrium - class 1
33405


In [99]:
# Create layer slider
select_layer = IntSlider(min=0, max=mri.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(layer):
    print(f"Plotting Layer: {layer}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 2, 1)
    plt.title("la_001", fontsize=20)
    plt.imshow(mri[:, :, layer], cmap='gray');
    plt.axis('off')
    
    fig.add_subplot(1, 2, 2)
    plt.title("Model prediction mask", fontsize=20)
    plt.imshow(prediction[:, :, layer], cmap='gray')
    plt.axis('off');

# Use the interactive() tool to create the visualization
interactive(plot, layer=select_layer)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Select Layer', max=179), Output…

#### Resizing to original size

In [100]:
prediction = prediction.astype(np.uint8)

# Transforming both image and mask at the same time yields different amount of labels
#  than doing them separately
transformed_mri = transform_original_dimensions(image=mri)
transformed_prediction = transform_original_dimensions(image=prediction)

original_size_mri = transformed_mri["image"].astype(np.float32)
original_size_prediction = transformed_prediction["image"].astype(np.uint8)

In [101]:
original_size_mri.shape

(400, 400, 180)

In [102]:
original_size_prediction.shape

(400, 400, 180)

In [103]:
# Number of labels per class in model prediction
print("Background - class 0")
print(len(original_size_prediction[original_size_prediction == 0])) 
print("--------")
print("Left Atrium - class 1")
print(len(original_size_prediction[original_size_prediction == 1]))

Background - class 0
28747979
--------
Left Atrium - class 1
52021


In [104]:
# Create layer slider
select_layer = IntSlider(min=0, max=original_size_mri.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(layer):
    print(f"Plotting Layer: {layer}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 2, 1)
    plt.title("la_001", fontsize=20)
    plt.imshow(original_size_mri[:, :, layer], cmap='gray');
    plt.axis('off')
    
    fig.add_subplot(1, 2, 2)
    plt.title("Model prediction mask", fontsize=20)
    plt.imshow(original_size_prediction[:, :, layer], cmap='gray')
    plt.axis('off');

# Use the interactive() tool to create the visualization
interactive(plot, layer=select_layer)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Select Layer', max=179), Output…